In [ ]:
# 현재 구글 드라이브에 저장
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re

In [ ]:
ing_1 = pd.read_excel('/content/drive/MyDrive/SSAFY 2학기_local/데이터/원자료/재료정보/식품의약품안전처/품목분류코드 전체자료_전처리_1.xlsx')
ing_2 = pd.read_excel('/content/drive/MyDrive/SSAFY 2학기_local/데이터/원자료/재료정보/식품의약품안전처/품목분류코드 전체자료_전처리_2.xlsx')

In [ ]:
extract_column = ['품목분류코드', '한글_명', '영문_명', '이명', '정의', '레벨', '상위_품목_코드','최상위_품목_코드','비고','종류']
ing_1 = ing_1[extract_column]
ing_2 = ing_2[extract_column]

In [ ]:
# 각각에서 임의로 추가한 행이 들어갈 수 있도록 outer join 수행
ing_df = pd.merge(ing_1, ing_2, on='품목분류코드', how='outer')

In [ ]:
ing_1.columns

Index(['품목분류코드', '한글_명', '영문_명', '이명', '정의', '레벨', '상위_품목_코드', '최상위_품목_코드',
       '비고', '종류'],
      dtype='object')

In [ ]:
ing_df.columns

Index(['품목분류코드', '한글_명_x', '영문_명_x', '이명_x', '정의_x', '레벨_x', '상위_품목_코드_x',
       '최상위_품목_코드_x', '비고_x', '종류_x', '한글_명_y', '영문_명_y', '이명_y', '정의_y',
       '레벨_y', '상위_품목_코드_y', '최상위_품목_코드_y', '비고_y', '종류_y'],
      dtype='object')

In [ ]:
def merged_column(df, nc, c1, c2):
  df[nc] = df.apply(lambda row: row[c1] if pd.notna(row[c1]) else row[c2], axis=1)

In [ ]:
merged_column_name = ['한글_명', '영문_명', '정의', '레벨', '상위_품목_코드', '최상위_품목_코드', '비고', '종류']

for column in merged_column_name:
  print(column+'_x')
  merged_column(ing_df, column, column+'_x', column+'_y')

ing_df['이명'] = ing_df.apply(lambda row: row['이명_x'] if pd.notna(row['이명_x']) and (len(str(row['이명_x'])) >= len(str(row['이명_y']))) else row['이명_y'], axis=1)

한글_명_x
영문_명_x
정의_x
레벨_x
상위_품목_코드_x
최상위_품목_코드_x
비고_x
종류_x


In [ ]:
merged_column_name.append('품목분류코드')
merged_column_name.append('이명')

In [ ]:
final_merged_df = ing_df[merged_column_name]

In [ ]:
null = final_merged_df[final_merged_df['한글_명'].isnull() | final_merged_df['레벨'].isnull() | final_merged_df['상위_품목_코드'].isnull() | final_merged_df['최상위_품목_코드'].isnull()]

In [ ]:
null

,한글_명,영문_명,정의,레벨,상위_품목_코드,최상위_품목_코드,비고,종류,품목분류코드,이명
50,찰벼,NaN,NaN,NaN,A0101500000000,A0000000000000,NaN,NaN,A0101500001813,NaN


In [ ]:
final_merged_df.at[50, '레벨'] = final_merged_df[final_merged_df['품목분류코드'] == null['상위_품목_코드'].values[0]]['레벨'].values[0]

In [ ]:
final_merged_df[final_merged_df['한글_명'].isnull() | final_merged_df['레벨'].isnull() | final_merged_df['상위_품목_코드'].isnull() | final_merged_df['최상위_품목_코드'].isnull()]

,한글_명,영문_명,정의,레벨,상위_품목_코드,최상위_품목_코드,비고,종류,품목분류코드,이명


In [ ]:
final_merged_df = final_merged_df[['품목분류코드','한글_명','영문_명','이명','정의','레벨','상위_품목_코드','최상위_품목_코드','비고','종류']]

In [ ]:
final_merged_df

,품목분류코드,한글_명,영문_명,이명,정의,레벨,상위_품목_코드,최상위_품목_코드,비고,종류
0,A0000000000000,식물성,NaN,NaN,NaN,1.0,A0000000000000,A0000000000000,NaN,NaN
1,A0100000000000,곡류,NaN,NaN,NaN,2.0,A0000000000000,A0000000000000,NaN,NaN
2,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반미', '청결미', '품질인증미', '혼합미', '흑미', '기타유색미', '향..."
3,A0100100001813,찹쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반계', '현미찹쌀', '기타'"
4,A0100100011813,찹쌀가루,NaN,찹쌀분말,NaN,4.0,A0100100001813,A0000000000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6725,Z3700100000000,인삼.홍삼제품,NaN,NaN,NaN,3.0,Z3700000000000,Z0000000000000,NaN,NaN
6726,Z3800000000000,기타기준규격외,NaN,NaN,NaN,2.0,Z0000000000000,Z0000000000000,NaN,NaN
6727,Z3800100000000,기타기준규격외,NaN,NaN,NaN,3.0,Z3800000000000,Z0000000000000,NaN,NaN
6728,Z3100200200813,젖소고기,NaN,NaN,NaN,5.0,Z3100200200000,Z0000000000000,NaN,"'윗등심살', '아래등심살', '꽃등심살', '살치살', '채끝', '목심', '앞..."


In [ ]:
final_merged_df.to_csv('ing.csv', encoding='utf-8-sig')

In [ ]:
def extractText(text):
  pattern = r"'"
  result = re.sub(pattern, '', text)

In [ ]:
final_merged_df = final_merged_df['품목분류코드','한글_명','영문명']

KeyError: ignored

In [ ]:
final_merged_df_origin = final_merged_df.copy()

In [ ]:
final_merged_df = final_merged_df_origin.copy()

In [ ]:
final_merged_df['복사됨'] = False

In [ ]:
not_null_rows = final_merged_df[final_merged_df['종류'].notnull()]
not_null_rows

,품목분류코드,한글_명,영문_명,이명,정의,레벨,상위_품목_코드,최상위_품목_코드,비고,종류,복사됨
2,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반미', '청결미', '품질인증미', '혼합미', '흑미', '기타유색미', '향...",False
3,A0100100001813,찹쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반계', '현미찹쌀', '기타'",False
5,A0100100002813,현미,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"기타', '현미쑥쌀', '메현미', '찰현미', '흑현미', '발아현미', '찰흑미...",False
9,A0100200000000,보리,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"'기타', '쌀보리', '맥주보리', '발아보리', '겉보리'",False
10,A0100200000813,보리쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"'겉보리쌀', '쌀보리쌀', '맥주보리쌀', '찰보리쌀', '늘보리쌀', '할맥',...",False
...,...,...,...,...,...,...,...,...,...,...,...
6569,Z3100500200000,육용계(닭),NaN,NaN,NaN,4.0,Z3100500000000,Z0000000000000,NaN,"육용종계', '개량종', '재래종', '기타 육용종계', '초생추', '개량종 육계...",False
6570,Z3100500300000,산란계(닭),NaN,NaN,NaN,4.0,Z3100500000000,Z0000000000000,NaN,"난용종계', '개량종', '재래종', '기타 난용종계', '초생추', '기타 산란계...",False
6575,Z3100600200000,오리,NaN,NaN,NaN,4.0,Z3100600000000,Z0000000000000,NaN,"오리', '종압(종오리)', '산란종', '육용종', '기타종', '육용 오리', ...",False
6728,Z3100200200813,젖소고기,NaN,NaN,NaN,5.0,Z3100200200000,Z0000000000000,NaN,"'윗등심살', '아래등심살', '꽃등심살', '살치살', '채끝', '목심', '앞...",False


In [ ]:

# 해당 행들을 바로 아래에 복사해서 삽입하면서 새 열 '복사됨'을 추가합니다.
for index, row in not_null_rows.iterrows():
    row_copy = row.copy()  # 해당 행의 복사본을 만듭니다.
    row_copy['복사됨'] = True  # 새 열 '복사됨'에 'True' 값을 할당합니다.
    final_merged_df.loc[index + 0.5] = row_copy  # 바로 아래에 새로운 행을 추가합니다.


In [ ]:
# 인덱스를 다시 정렬합니다.
final_merged_df.sort_index(inplace=True)

In [ ]:
final_merged_df.reset_index(drop=True, inplace=True)

In [ ]:
final_merged_df

,품목분류코드,한글_명,영문_명,이명,정의,레벨,상위_품목_코드,최상위_품목_코드,비고,종류,복사됨
0,A0000000000000,식물성,NaN,NaN,NaN,1.0,A0000000000000,A0000000000000,NaN,NaN,False
1,A0100000000000,곡류,NaN,NaN,NaN,2.0,A0000000000000,A0000000000000,NaN,NaN,False
2,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반미', '청결미', '품질인증미', '혼합미', '흑미', '기타유색미', '향...",False
3,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반미', '청결미', '품질인증미', '혼합미', '흑미', '기타유색미', '향...",True
4,A0100100001813,찹쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반계', '현미찹쌀', '기타'",False
...,...,...,...,...,...,...,...,...,...,...,...
6989,Z3800100000000,기타기준규격외,NaN,NaN,NaN,3.0,Z3800000000000,Z0000000000000,NaN,NaN,False
6990,Z3100200200813,젖소고기,NaN,NaN,NaN,5.0,Z3100200200000,Z0000000000000,NaN,"'윗등심살', '아래등심살', '꽃등심살', '살치살', '채끝', '목심', '앞...",False
6991,Z3100200200813,젖소고기,NaN,NaN,NaN,5.0,Z3100200200000,Z0000000000000,NaN,"'윗등심살', '아래등심살', '꽃등심살', '살치살', '채끝', '목심', '앞...",True
6992,Z3100200300813,육우고기,NaN,NaN,NaN,4.0,Z3100200000000,Z0000000000000,NaN,"젖소', '젖소종축', '홀스타인', '저어지', '거언지', '에어셔', '유용쇼...",False


In [ ]:
final_merged_df['종류'] = final_merged_df['종류'].str.replace(r"\s+|'", '', regex=True)

In [ ]:
final_merged_df

,품목분류코드,한글_명,영문_명,이명,정의,레벨,상위_품목_코드,최상위_품목_코드,비고,종류,복사됨
0,A0000000000000,식물성,NaN,NaN,NaN,1.0,A0000000000000,A0000000000000,NaN,NaN,False
1,A0100000000000,곡류,NaN,NaN,NaN,2.0,A0000000000000,A0000000000000,NaN,NaN,False
2,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반미,청결미,품질인증미,혼합미,흑미,기타유색미,향미,양조미,대립미,쑥쌀,멥쌀,기타",False
3,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반미,청결미,품질인증미,혼합미,흑미,기타유색미,향미,양조미,대립미,쑥쌀,멥쌀,기타",True
4,A0100100001813,찹쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반계,현미찹쌀,기타",False
...,...,...,...,...,...,...,...,...,...,...,...
6989,Z3800100000000,기타기준규격외,NaN,NaN,NaN,3.0,Z3800000000000,Z0000000000000,NaN,NaN,False
6990,Z3100200200813,젖소고기,NaN,NaN,NaN,5.0,Z3100200200000,Z0000000000000,NaN,"윗등심살,아래등심살,꽃등심살,살치살,채끝,목심,앞다리,꾸리살,갈비덧살,부채살,앞다리...",False
6991,Z3100200200813,젖소고기,NaN,NaN,NaN,5.0,Z3100200200000,Z0000000000000,NaN,"윗등심살,아래등심살,꽃등심살,살치살,채끝,목심,앞다리,꾸리살,갈비덧살,부채살,앞다리...",True
6992,Z3100200300813,육우고기,NaN,NaN,NaN,4.0,Z3100200000000,Z0000000000000,NaN,"젖소,젖소종축,홀스타인,저어지,거언지,에어셔,유용쇼오트혼,브라운스위스,기타젖소,젖소...",False


In [ ]:
final_merged_df.loc[~final_merged_df['복사됨'], '종류'] = None

In [ ]:
final_merged_df

,품목분류코드,한글_명,영문_명,이명,정의,레벨,상위_품목_코드,최상위_품목_코드,비고,종류,복사됨
0,A0000000000000,식물성,NaN,NaN,NaN,1.0,A0000000000000,A0000000000000,NaN,None,False
1,A0100000000000,곡류,NaN,NaN,NaN,2.0,A0000000000000,A0000000000000,NaN,None,False
2,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,None,False
3,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,"일반미,청결미,품질인증미,혼합미,흑미,기타유색미,향미,양조미,대립미,쑥쌀,멥쌀,기타",True
4,A0100100001813,찹쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,None,False
...,...,...,...,...,...,...,...,...,...,...,...
6989,Z3800100000000,기타기준규격외,NaN,NaN,NaN,3.0,Z3800000000000,Z0000000000000,NaN,None,False
6990,Z3100200200813,젖소고기,NaN,NaN,NaN,5.0,Z3100200200000,Z0000000000000,NaN,None,False
6991,Z3100200200813,젖소고기,NaN,NaN,NaN,5.0,Z3100200200000,Z0000000000000,NaN,"윗등심살,아래등심살,꽃등심살,살치살,채끝,목심,앞다리,꾸리살,갈비덧살,부채살,앞다리...",True
6992,Z3100200300813,육우고기,NaN,NaN,NaN,4.0,Z3100200000000,Z0000000000000,NaN,None,False


In [ ]:
# final_merged_df['종류'] = final_merged_df['종류'].str.split(",")
final_merged_df['종류'] = final_merged_df['종류'].fillna('').apply(lambda x: [item.strip() for item in x.split(',') if item.strip() != '' and '기타' not in item])

In [ ]:

df_exploded = final_merged_df.explode('종류', ignore_index=True)

print(df_exploded)

              품목분류코드  한글_명 영문_명   이명   정의   레벨        상위_품목_코드  \
0     A0000000000000   식물성  NaN  NaN  NaN  1.0  A0000000000000   
1     A0100000000000    곡류  NaN  NaN  NaN  2.0  A0000000000000   
2     A0100100000000     쌀  NaN  NaN  NaN  3.0  A0100000000000   
3     A0100100000000     쌀  NaN  NaN  NaN  3.0  A0100000000000   
4     A0100100000000     쌀  NaN  NaN  NaN  3.0  A0100000000000   
...              ...   ...  ...  ...  ...  ...             ...   
9238  Z3100200300813  육우고기  NaN  NaN  NaN  4.0  Z3100200000000   
9239  Z3100200300813  육우고기  NaN  NaN  NaN  4.0  Z3100200000000   
9240  Z3100200300813  육우고기  NaN  NaN  NaN  4.0  Z3100200000000   
9241  Z3100200300813  육우고기  NaN  NaN  NaN  4.0  Z3100200000000   
9242  Z3100200300813  육우고기  NaN  NaN  NaN  4.0  Z3100200000000   

           최상위_품목_코드   비고     종류    복사됨  
0     A0000000000000  NaN    NaN  False  
1     A0000000000000  NaN    NaN  False  
2     A0000000000000  NaN    NaN  False  
3     A0000000000000  NaN    일반미   True

In [ ]:
df_exploded

,품목분류코드,한글_명,영문_명,이명,정의,레벨,상위_품목_코드,최상위_품목_코드,비고,종류,복사됨
0,A0000000000000,식물성,NaN,NaN,NaN,1.0,A0000000000000,A0000000000000,NaN,NaN,False
1,A0100000000000,곡류,NaN,NaN,NaN,2.0,A0000000000000,A0000000000000,NaN,NaN,False
2,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,NaN,False
3,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,일반미,True
4,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,청결미,True
...,...,...,...,...,...,...,...,...,...,...,...
9238,Z3100200300813,육우고기,NaN,NaN,NaN,4.0,Z3100200000000,Z0000000000000,NaN,안심,True
9239,Z3100200300813,육우고기,NaN,NaN,NaN,4.0,Z3100200000000,Z0000000000000,NaN,등심,True
9240,Z3100200300813,육우고기,NaN,NaN,NaN,4.0,Z3100200000000,Z0000000000000,NaN,윗등심살,True
9241,Z3100200300813,육우고기,NaN,NaN,NaN,4.0,Z3100200000000,Z0000000000000,NaN,아래등심살,True


In [ ]:
# '복사됨'이 True인 경우 한글_명 데이터를 종류 데이터로, 상위_품목_코드를 품목분류코드로, 레벨을 +1 증가시킵니다.
df_exploded.loc[df_exploded['복사됨'], '한글_명'] = df_exploded.loc[df_exploded['복사됨'], '종류']
df_exploded.loc[df_exploded['복사됨'], '상위_품목_코드'] = df_exploded.loc[df_exploded['복사됨'], '품목분류코드']
df_exploded.loc[df_exploded['복사됨'], '레벨'] = df_exploded.loc[df_exploded['복사됨'], '레벨'] + 1

# '복사됨' 열은 더 이상 필요하지 않으므로 삭제합니다.
df_exploded.drop('복사됨', axis=1, inplace=True)


In [ ]:
df_exploded

,품목분류코드,한글_명,영문_명,이명,정의,레벨,상위_품목_코드,최상위_품목_코드,비고,종류
0,A0000000000000,식물성,NaN,NaN,NaN,1.0,A0000000000000,A0000000000000,NaN,NaN
1,A0100000000000,곡류,NaN,NaN,NaN,2.0,A0000000000000,A0000000000000,NaN,NaN
2,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,NaN
3,A0100100000000,일반미,NaN,NaN,NaN,4.0,A0100100000000,A0000000000000,NaN,일반미
4,A0100100000000,청결미,NaN,NaN,NaN,4.0,A0100100000000,A0000000000000,NaN,청결미
...,...,...,...,...,...,...,...,...,...,...
9238,Z3100200300813,안심,NaN,NaN,NaN,5.0,Z3100200300813,Z0000000000000,NaN,안심
9239,Z3100200300813,등심,NaN,NaN,NaN,5.0,Z3100200300813,Z0000000000000,NaN,등심
9240,Z3100200300813,윗등심살,NaN,NaN,NaN,5.0,Z3100200300813,Z0000000000000,NaN,윗등심살
9241,Z3100200300813,아래등심살,NaN,NaN,NaN,5.0,Z3100200300813,Z0000000000000,NaN,아래등심살


In [ ]:
df_exploded.dropna(subset=['한글_명'], inplace=True)

In [ ]:
df_exploded

,품목분류코드,한글_명,영문_명,이명,정의,레벨,상위_품목_코드,최상위_품목_코드,비고,종류
0,A0000000000000,식물성,NaN,NaN,NaN,1.0,A0000000000000,A0000000000000,NaN,None
1,A0100000000000,곡류,NaN,NaN,NaN,2.0,A0000000000000,A0000000000000,NaN,None
2,A0100100000000,쌀,NaN,NaN,NaN,3.0,A0100000000000,A0000000000000,NaN,None
3,A0100100000000,일반미,NaN,NaN,NaN,4.0,A0100100000000,A0000000000000,NaN,일반미
4,A0100100000000,청결미,NaN,NaN,NaN,4.0,A0100100000000,A0000000000000,NaN,청결미
...,...,...,...,...,...,...,...,...,...,...
9367,Z3100200300813,등심,NaN,NaN,NaN,5.0,Z3100200300813,Z0000000000000,NaN,등심
9368,Z3100200300813,윗등심살,NaN,NaN,NaN,5.0,Z3100200300813,Z0000000000000,NaN,윗등심살
9369,Z3100200300813,아래등심살,NaN,NaN,NaN,5.0,Z3100200300813,Z0000000000000,NaN,아래등심살
9370,Z3100200300813,꽃등심살,NaN,NaN,NaN,5.0,Z3100200300813,Z0000000000000,NaN,꽃등심살


In [ ]:
df_exploded.to_csv('final_ing.csv', encoding='utf-8-sig')